In [18]:
from multiprocessing import freeze_support
import pandas as pd
import string
import nltk
import re
import matplotlib.pyplot as plt
import gensim.corpora as corpora
import gensim
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from pprint import pprint
import numpy as np
from gensim.models import Phrases, CoherenceModel
from nltk.tokenize import word_tokenize
from gensim.test.utils import datapath
from sklearn.cluster import MiniBatchKMeans, KMeans
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.cm as cm
from sklearn.manifold import TSNE
from gensim.corpora import Dictionary
import pickle
from gensim.test.utils import datapath

nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')
nltk.download('wordnet')
nltk.download('punkt')

wordnet_lemmatizer = WordNetLemmatizer()
wpt = nltk.WordPunctTokenizer()
lemmatizer = nltk.WordNetLemmatizer()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\zesha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\zesha\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\zesha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [19]:
def createList(corpus):
    corpus_list = []
    for i in range (len(corpus)):
        temp = []
        for word in (corpus[i]):
            temp.append(word)
        temp = " ".join(temp)
        corpus_list.append(temp)
    return corpus_list

In [20]:
def find_optimal_clusters(dataframe1, max_k):
    iters = range(1, max_k+1, 1)
    
    sse = []
    for k in iters:
        sse.append(MiniBatchKMeans(n_clusters=k, init_size=1024, batch_size=2048, random_state=100).fit(dataframe1).inertia_)
        print('Fit {} clusters'.format(k))
        
    f, ax = plt.subplots(1, 1)
    ax.plot(iters, sse, marker='o')
    ax.set_xlabel('Cluster Centers')
    ax.set_xticks(iters)
    ax.set_xticklabels(iters)
    ax.set_ylabel('SSE')
    ax.set_title('SSE by Cluster Center Plot')

In [21]:
def plot_tsne_pca(dataframe1, labels):
    max_label = max(labels)
    max_items = np.random.choice(range(dataframe1.shape[0]), size=10000, replace=False)
    
    pca = PCA(n_components=2).fit_transform(dataframe1[max_items,:].todense())
    tsne = TSNE().fit_transform(PCA(n_components=50).fit_transform(dataframe1[max_items,:].todense()))    
    
    idx = np.random.choice(range(pca.shape[0]), size=300, replace=False)
    label_subset = labels[max_items]
    label_subset = [cm.hsv(i/max_label) for i in label_subset[idx]]
    
    f, ax = plt.subplots(1, 2, figsize=(14, 6))
    
    ax[0].scatter(pca[idx, 0], pca[idx, 1], c=label_subset)
    ax[0].set_title('PCA Cluster Plot')
    
    ax[1].scatter(tsne[idx, 0], tsne[idx, 1], c=label_subset)
    ax[1].set_title('TSNE Cluster Plot')

In [1]:
def similarity(resume_topics, job_topics):
    
    weights = [0.25, 0.18, 0.15, 0.13, 0.11, 0.09, 0.05, 0.04]
    distribution = [1, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3]
    
    accuracy_lst = []
    for topic in resume_topics:
        if topic not in job_topics:
            accuracy_lst.append(0)
        else:
            current_weight = weights[resume_topics.index(topic)]
            job_index = job_topics.index(topic)
            resume_index = resume_topics.index(topic)
            temp = abs(job_index - resume_index)
            distrib_num = distribution[temp]
            accuracy_lst.append(distrib_num*current_weight)

    return (((round((sum(accuracy_lst)),2)*100)))